In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install laserembeddings

     |████████████████████████████████| 51kB 2.6MB/s 
     |████████████████████████████████| 860kB 4.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=6b210b4558c7207f2050885bb2e1a6669287434c7e44838684d001f66fbb05ab
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [9]:
!python -m laserembeddings download-models


✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


In [15]:
import pandas as pd
import numpy as np
import os
from laserembeddings import Laser
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import pickle

In [11]:
def preprocessing(language):
  path = 'drive/My Drive/CS695'
  data = pd.read_csv(os.path.join(path,language+'.csv'))
  data_to_list = data['tweet'].to_list()

  laser = Laser()
  embeddings = laser.embed_sentences(data_to_list, lang='en') 

  df1=pd.DataFrame(embeddings)
  df2 = pd.DataFrame(data['label'])

  sss1 = StratifiedShuffleSplit(n_splits=1,test_size=0.3,random_state=0)
  a = list(sss1.split(df1, df2))
  train_x,interX = df1.iloc[a[0][0],:],df1.iloc[a[0][1],:]
  train_y,interY = df2.iloc[a[0][0],:],df2.iloc[a[0][1],:]

  sss2 = StratifiedShuffleSplit(n_splits=1,test_size=0.35,random_state=0)
  a = list(sss2.split(interX, interY))
  test_x,val_x = interX.iloc[a[0][0],:],interX.iloc[a[0][1],:]
  test_y,val_y = interY.iloc[a[0][0],:],interY.iloc[a[0][1],:]

  return train_x, test_x, val_x, train_y, test_y, val_y 

In [17]:
def model_for_sample_data(language, sample, c, seed):

  train_x, test_x, val_x, train_y, test_y, val_y = preprocessing(language)

  sss = StratifiedShuffleSplit(n_splits=1,test_size= sample ,random_state=seed)
  a = list(sss.split(train_x, train_y))
  train_x,train_y = train_x.iloc[a[0][1],:],train_y.iloc[a[0][1],:]

  model = LogisticRegression(C=c,solver='lbfgs',class_weight='balanced',random_state=seed)
  model.fit(train_x, train_y)

  test_predict = model.predict(test_x)
  val_predict = model.predict(val_x)

  test_f1_score = f1_score(test_y, test_predict, average='macro')
  val_f1_score = f1_score(val_y, val_predict, average='macro')
  test_acc = accuracy_score(test_y, test_predict)
  val_acc = accuracy_score(val_y, val_predict)

  #saved_model = pickle.dumps(model) 

  print('Language:',language,'    ', 'Data loaded: sample data','    ', 'sample_size:',sample,'    ','c:',c,'    ', 'seed:',seed)
  print('\n F1 macro score for test:', test_f1_score)
  print('\n F1 macro score for val:', val_f1_score)
  print('\n Accuracy for test:', test_acc)
  print('\n Accuracy for val:', val_acc)

In [18]:
def model_for_whole_data(language, c, seed):

  train_x, test_x, val_x, train_y, test_y, val_y = preprocessing(language)

  model = LogisticRegression(C=c,solver='lbfgs',class_weight='balanced',random_state=seed)
  model.fit(train_x, train_y)

  test_predict = model.predict(test_x)
  val_predict = model.predict(val_x)

  test_f1_score = f1_score(test_y, test_predict, average='macro')
  val_f1_score = f1_score(val_y, val_predict, average='macro')
  test_acc = accuracy_score(test_y, test_predict)
  val_acc = accuracy_score(val_y, val_predict)

  #saved_model = pickle.dumps(model) 

  print('Language:',language,'     ' ,'data loaded: whole data', '    ','c:',c,'    ', 'seed:',seed)
  print('\n F1 macro score for test:', test_f1_score)
  print('\n F1 macro score for val:', val_f1_score)
  print('\n Accuracy for test:', test_acc)
  print('\n Accuracy for val:', val_acc)

In [19]:
for language in ['Arabic','English','French']:
      for sample in [16,32,64,128,256,'full']:
        for c in [0.01,0.1,1,10]:
          for seed in [2018,2019,2020,2021,2022]:
            np.random.seed(seed)
            if(sample == 'full'):
              model_for_whole_data(language, c, seed)
            else:
              model_for_sample_data(language, sample, c, seed)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Language: Arabic      Data loaded: sample data      sample_size: 16      c: 0.01      seed: 2018

 F1 macro score for test: 0.5569516389114831

 F1 macro score for val: 0.5748381016083427

 Accuracy for test: 0.8051372896368467

 Accuracy for val: 0.8108552631578947


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Language: Arabic      Data loaded: sample data      sample_size: 16      c: 0.01      seed: 2019

 F1 macro score for test: 0.5156484207418215

 F1 macro score for val: 0.5146589259796808

 Accuracy for test: 0.7812223206377326

 Accuracy for val: 0.7828947368421053


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Language: French       data loaded: whole data      c: 10      seed: 2020

 F1 macro score for test: 0.6163178528935613

 F1 macro score for val: 0.6312080313808922

 Accuracy for test: 0.6497890295358649

 Accuracy for val: 0.6666666666666666


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Language: French       data loaded: whole data      c: 10      seed: 2021

 F1 macro score for test: 0.6163178528935613

 F1 macro score for val: 0.6312080313808922

 Accuracy for test: 0.6497890295358649

 Accuracy for val: 0.6666666666666666
Language: French       data loaded: whole data      c: 10      seed: 2022

 F1 macro score for test: 0.6163178528935613

 F1 macro score for val: 0.6312080313808922

 Accuracy for test: 0.6497890295358649

 Accuracy for val: 0.6666666666666666


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
